In [2]:
import numpy as np
from numpy import random
import random
import matplotlib.pyplot as plt
from scipy.signal import cwt,ricker
from scipy import stats
from sklearn.preprocessing import StandardScaler,normalize
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import KFold
import os
import tensorflow as tf
import tensorflow.keras as keras
import tensorflow.keras.optimizers
from tensorflow.keras.models import Sequential
from keras.callbacks import EarlyStopping
from keras.models import load_model
from tensorflow.keras.layers import Flatten,concatenate,Layer,Dense,LSTM,Activation,MaxPooling2D,Dropout,Conv2D,BatchNormalization,Reshape,UpSampling2D,ZeroPadding2D
import radiomics
from radiomics.featureextractor import RadiomicsFeatureExtractor
import SimpleITK as sitk
import pandas as pd


# Example: Load An Image

Example how to load an image with label

In [100]:
# the folder name by label
class_names = ['pos','neg']

# the image dictionary we are loading
dict = './covid19/kcv/'

In [101]:
# load one image with their label to memory
def load_image( imageName ):

    for i in class_names:
        path = os.path.join(dict, i)
        label_num = class_names.index(i)
        try:
            img = sitk.ReadImage(dict + i +'/' + imageName)
            trainingdata= [sitk.GetArrayFromImage(img).reshape(512,512,1),label_num]
        except:
            pass
            
    return trainingdata
    

In [102]:
singleImage = load_image('COVID-19_0414.png')

In [103]:
# get the label and the image
image = singleImage[0]
label = singleImage[1]

In [104]:
print(label)

0


# Example: Extract the Features From a Image

Example how to extract the Features using pyradiomics library applied in a whole image

In [79]:
# the image dictionary and the image name want to extract feature from
dict = './covid19/kcv/pos/'

def ExtractFeaturesFromImages(imageName):
    
    feature_extracted = []

    extractor = RadiomicsFeatureExtractor()
    # Disable all classes 
    extractor.disableAllFeatures()
    extractor.enableFeaturesByName(firstorder=['Skewness'])  # change here to extract

    im = sitk.ReadImage(dict + imageName)

    ma_arr = np.ones(im.GetSize()[::-1])  # reverse the order as image is xyz, array is zyx
    ma = sitk.GetImageFromArray(ma_arr)
    ma.CopyInformation(im)  # Copy geometric info

    # extract features and fill the output array
    features = extractor.execute(im, ma)

    feature_extracted.append(features['original_firstorder_Skewness'])
      
    return feature_extracted


In [80]:
feature_extracted = ExtractFeaturesFromImages('COVID-19_0414.png')

# Example: To Train A Fin

When we have images and corrresponding feature extracted, we can train a DFNN to iminate these features

In [ ]:
def MakeFin(modelname):
    input_layer = keras.Input(shape=(512,512,1))
    x = Flatten()(input_layer)
    x = Dense(64, activation='relu')(x)
    x = Dense(64, activation='relu')(x)
    x = Dense(64, activation='relu')(x)

    out = Dense(1, activation = 'linear', name= modelname)(x)

    model = keras.Model(input_layer, out, name= modelname)
    
    return model

In [ ]:
model = MakeFin('Name')

In [ ]:
model.compile(optimizer=tf.keras.optimizers.Adam(
), loss="mse", metrics='mae')

In [ ]:
model.fit(input_array, output_array, 
              epochs=30, 
          batch_size=64,
                shuffle=True
            )

# Example: Load A Fin and Apply in a Image

Example we load an image into numpy array, and we use Fin to predict the feature of the image

In [95]:
image = np.array(image)

In [96]:
print(image.shape)

(512, 512, 1)


In [97]:
fin = load_model('finetune_Autocorrelation.h5')

In [87]:
inputimage = image.reshape(1,512,512,1)

In [99]:
fin.predict(inputimage)

1/1 [==============================] - 0s 30ms/step


array([[42.890038]], dtype=float32)

# Example Use the FINs in another Network

Load the FINs and attached to a DFNN

In [ ]:
# Load two Fins
fin1 = load_model('finetune_Skewness.h5')
fin2 = load_model('finetune_Autocorrelation.h5')


def MakeModel(modelname):
    input_layer = keras.Input(shape=(512,512,1))
    
    # load models and concat them
    l1 = fin1(input_layer)
    l2 = fin2(input_layer)
    
    n_d = concatenate([l1,l2])

    x = Dense(128, activation='relu',name='x1')(n_d)

    out = Dense(2, activation = 'softmax', name= modelname)(x)

    model = keras.Model(input_layer, out, name= modelname)
    return model